# Analysis of web server logs with Spark

In [1]:
import os
import sys #current as of 9/26/2015
import pyspark
from pyspark.sql import SQLContext
# We can give a name to our app (to find it in Spark WebUI) and configure execution mode
# In this case, it is local multicore execution with "local[*]"
app_name = "example-logs"
master = "local[*]"
conf = pyspark.SparkConf().setAppName(app_name).setMaster(master)
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)
print(sc)
print(sqlContext)
# Import some libraries to work with dates
import dateutil.parser
import dateutil.relativedelta as dateutil_rd

# the data per country
date and time| address| country| Success or failure

__DATA__
The data for this quiz is located at https://www.dropbox.com/s/ul1shmad5rdy29m/example.log?dl=0

In [2]:
!head ./data/example.log 

2015-12-10T13:36:47|86.88.135.133|FR|SUCCESS
2015-12-10T13:36:47|25.53.251.132|FR|SUCCESS
2015-12-10T13:36:47|224.251.26.196|FR|SUCCESS
2015-12-10T13:36:47|72.96.92.96|FR|SUCCESS
2015-12-10T13:36:47|93.159.208.231|FR|SUCCESS
2015-12-10T13:36:47|200.85.209.180|FR|SUCCESS
2015-12-10T13:36:47|117.35.65.164|FR|SUCCESS
2015-12-10T13:36:47|164.86.143.136|FR|SUCCESS
2015-12-10T13:36:47|56.160.244.16|FR|SUCCESS
2015-12-10T13:36:47|205.111.122.245|FR|SUCCESS


In [3]:
# Read log file
logs = sc.textFile("./data/example.log")

# Print total number of lines
logs.count()

# Mostrar las 15 primeras lineas
logs.take(15)

# Print total lenght of file (chars)
line_len = logs.map(lambda s: len(s))
total_len = line_len.reduce(lambda a, b: a + b)
print(total_len)

31210897


In [4]:
# Separate items from each line and store results in cache
# to speed up subsequent access
line_items = logs.map(lambda line: line.split("|"))
line_items.cache()

PythonRDD[5] at RDD at PythonRDD.scala:43

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib inline


def plot(delays, measureStr):
    """
    Show a bar chart of the total delay per airline
    """
    airlines = [d[0] for d in delays]
    minutes  = [d[1] for d in delays]
    index    = list(xrange(len(airlines)))

    fig, axe = plt.subplots()
    bars = axe.barh(index, minutes)

    # Add the total minutes to the right
    for idx, air, min in zip(index, airlines, minutes):
        if min > 0:
            bars[idx].set_color('#d9230f')
            axe.annotate(" %0.0f min" % min, xy=(min+1, idx+0.5), va='center')
        else:
            bars[idx].set_color('#469408')
            axe.annotate(" %0.0f" % min, xy=(10, idx+0.5), va='center')

    # Set the ticks
    ticks = plt.yticks([idx+ 0.5 for idx in index], airlines)
    xt = plt.xticks()[0]
    plt.xticks(xt, [' '] * len(xt))

    #minimize chartjunk
    plt.grid(axis = 'x', color ='white', linestyle='-')

    plt.title(measureStr)
    plt.show()



# QUIZ:  Exploratory Data Analysis

In [25]:
# How many log entries by  country
# Format of data is as follows:   date and time| address| country| Success or failure
logs = sc.textFile("./data/example.log")
counts = logs.map(lambda line: line.split("|")[2])

#How many countries
cnt = 0
for i in counts.distinct().collect():
    cnt += 1
print (cnt)
#plot top 10 countries freiq
#plot(logFileCountsPerCountry[:10], "Number of logfile entries per country")


230


In [31]:
# How many log entries by  country
# Format of data is as follows:   date and time| address| country| Success or failure
logs = sc.textFile("./data/example.log")
logFileCountsPerCountry = logs.map(lambda line: line.split("|")[2]) \
    .map(lambda country: (country, 1)) \
    .reduceByKey(lambda a, b: a + b, 1) \
    .sortBy(lambda a: -a[1])

#How many countries
cnt = 0
for i in logFileCountsPerCountry.distinct().collect():
    cnt += 1
print (cnt)
#plot top 10 countries freiq
plot(logFileCountsPerCountry[:10], "Number of logfile entries per country")


230


TypeError: 'PipelinedRDD' object is not subscriptable

In [10]:
line = "date and time| address| country| Success or failure"
print (line.split("|")[2])

 country


In [17]:
cnt = 0
print (cnt)

0
